# Query the Dataset in S3 using Amazon Athena and AWS Glue Catalog
This will create an Athena table in the Glue Catalog (Hive Metastore) which allows us to query the dataset files in S3.  We will create a table in the Glue Catalog based on the `Amazon Customer Reviews Dataset` in S3.  We define the columns that map to the data, specify how the data is formatted, and provide the location in Amazon S3 for the file(s).


<img src="img/athena_register_parquet.png" width="60%" align="left">

In [1]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

In [2]:
from pyathena import connect
import pandas as pd

# Create Athena Table for Parquet Data

#### Dataset columns

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.



In [3]:
s3_staging_dir = "s3://{}/athena/query-cache".format(bucket)

In [4]:
database_name = "default"
table_name = "amazon_reviews_parquet"

In [5]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [6]:
# SQL statement to execute
statement = """
    CREATE EXTERNAL TABLE {}.{}(
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date bigint, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://amazon-reviews-pds/parquet/'
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)


    CREATE EXTERNAL TABLE default.amazon_reviews_parquet(
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date bigint, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://amazon-reviews-pds/parquet/'



""


# Analyze and Build the Partitions

In [7]:
statement = """
    MSCK REPAIR TABLE {}.{}
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)


    MSCK REPAIR TABLE default.amazon_reviews_parquet



""


# Verify The Table Has Been Created Succesfully

In [8]:
statement = "SHOW TABLES in {}".format(database_name)

df_tables = pd.read_sql(statement, conn)
df_tables

,tab_name
0,amazon_reviews_parquet


# Run A Sample Query

In [9]:
product_category = "Digital_Software"

statement = """
    SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100
""".format(
    database_name, table_name, product_category
)

print(statement)


    SELECT * FROM default.amazon_reviews_parquet
    WHERE product_category = 'Digital_Software' LIMIT 100



In [10]:
df = pd.read_sql(statement, conn)
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,year,product_category
0,US,16439457,RTP8B70WFVEAB,B00FFINFM2,218020461,TurboTax Business Fed and Efile 2013,2,0,0,N,Y,Worse than their products for Individuals,Maybe I just had too many expectations.<br />I...,16170,2014,Digital_Software
1,US,18171562,R1XEO4NYQ0RD4Y,B009G6SEKM,440579914,Adobe Photoshop Elements 11,5,1,1,N,Y,Very good software,Both programs perform excellently and APE11 se...,15898,2013,Digital_Software
2,US,10787611,R1X65MS9H1YXZY,B00H9A60O4,608720080,Avast Free Antivirus 2015 [Download],1,0,0,N,Y,Didn't catch any viruses or malware,After downloading it on my son's computer it d...,16170,2014,Digital_Software
3,US,6676531,R18LC1I0XMW6HA,B008SCNLEY,643150354,Norton 360 2013 - 1 User / 3 PC,5,2,2,N,N,Renewing Norton 360,I had problems trying to download and install ...,15898,2013,Digital_Software
4,US,29435045,R2JF1T6Z3R91ZV,B00G0DXA9Y,595906078,H&R Block At Home 2013 Deluxe + State,4,0,0,N,Y,"User Friendly, would recommend.",Find this software is very user friendly. One...,16170,2014,Digital_Software


In [11]:
if df.empty:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA.")
    print("LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.             ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [12]:
if "amazon_reviews_parquet" in df_tables.values and not df.empty:
    ingest_create_athena_table_parquet_passed = True

In [13]:
%store ingest_create_athena_table_parquet_passed

Stored 'ingest_create_athena_table_parquet_passed' (bool)


# Review the New Athena Table in the Glue Catalog

In [14]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#table:catalog={};name=amazon_reviews_parquet;namespace=default">AWS Glue Catalog</a></b>'.format(
            region, account_id
        )
    )
)

# Store Variables for the Next Notebooks

In [15]:
%store

Stored variables and their in-db values:
ingest_create_athena_table_parquet_passed             -> True
setup_dependencies_passed                             -> True


# Release Resources

In [16]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>